# core

> the core functionalities of fastdebug

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

## make life easier with defaults  

In [ ]:
#| export
defaults = type('defaults', (object,), {'margin': 157 # align to the right by 157
                                        # 'eg': None, # examples
                                        # 'src': None, # official src
                                       }) 

## Execute strings

In [ ]:
eval?

Signature: eval(source, globals=None, locals=None, /)
Docstring:
Evaluate the given source in the context of globals and locals.

The source may be a string representing a Python expression
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


In [ ]:
exec?

Signature: exec(source, globals=None, locals=None, /)
Docstring:
Execute the given source in the context of globals and locals.

The source may be a string representing one or more Python statements
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


In [ ]:
#| export
from pprint import pprint

### eval can access from globals() and locals()

In [ ]:
x = 1
def test():
    a = "1+x"
    pprint(f'locals: {locals()}', width=157) # x is not in locals() but in globals()
    print(eval(a)) 
test()

"locals: {'a': '1+x'}"
2


### eval can override its own globals() and locals()

In [ ]:
def test():
    a = 1
    b = "a+1"
    pprint(f'locals: {locals()}', width=157)
    print(f'b: {eval(b, {}, {"a": 2})}') # globals() put to empty and locals() to include 'a' with a different value
    pprint(f'locals: {locals()}', width=157) 

test()

"locals: {'a': 1, 'b': 'a+1'}"
b: 3
"locals: {'a': 1, 'b': 'a+1'}"


In [ ]:
def test():
    a = 1
    b = "c = 1 + 1"
    exec(b)
    pprint(f'locals: {locals()}', width=157) # c is created and stored in locals()
    print(a + c) # but c is not accessible directly

try:
    test()
except NameError as e:
    print(e)

"locals: {'a': 1, 'b': 'c = 1 + 1', 'c': 2}"
name 'c' is not defined


In [ ]:
def test():
    a = 1
    b = "c = 1 + 1"
    exec(b)
    pprint(f'locals: {locals()}', width=157) # c is created and stored in locals()
    d = locals()['c'] # we can't assign the value 2 back to c here, but have to choose a different variable
    print(a + d)

test()

"locals: {'a': 1, 'b': 'c = 1 + 1', 'c': 2}"
3


In [ ]:
def test():
    a = 1
    b = "def add(x, y):\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    print(f'b: {eval(b)}')
try:
    test()
except SyntaxError as e:
    print(e)

"locals: {'a': 1, 'b': 'def add(x, y):\\n    return x + y'}"
invalid syntax (<string>, line 1)


In [ ]:
def test():
    a = 1
    b = "def add(x, y):\n    return x + y"
    exec(b) # it returns nothing, but add function is created and stored inside locals()
    pprint(f'locals: {locals()}', width=157)
    print(add(1, 9)) # can't access add from locals() directly

try:
    test()
except NameError as e:
    print(e)

"locals: {'a': 1, 'b': 'def add(x, y):\\n    return x + y', 'add': <function add>}"
name 'add' is not defined


In [ ]:
def test():
    a = 1
    b = "def add(x, y):\n    return x + y"
    print(f'b: {exec(b)}')
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] # doing add = locals()['add'] will cause error
    print(add1(1,9))

test()

b: None
"locals: {'a': 1, 'b': 'def add(x, y):\\n    return x + y', 'add': <function add>}"
10


### Be careful with the naming of the function you are overriding with string

In [ ]:
def add(x, y): pass

def test(func):
    a = 1
    b = "def add(x, y):\n    return x + y"
    c = func
    exec(b)
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] # doing add = locals()['add'] will cause error
    print(add1(1,9))

test(add)

("locals: {'func': <function add>, 'a': 1, 'b': 'def add(x, y):\\n    return x + y', 'c': <function add>, 'add': <function "
 'add at 0x11dcc3430>}')
10


In [ ]:
def add(x, y): pass

def test(add): # change func to add: cause an error! so, naming carefully and sensibly is important
    a = 1
    b = "def add(x, y):\n    return x + y"
    c = add  # change func to add: cause an error
    exec(b) # in this example, {'add': <function 'add at 0x1184f7e50>}' does not get created and stored in locals()
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] # doing add = locals()['add'] will cause error
    print(add1(1,9))

try:
    test(add)
except TypeError as e:
    print(e)

"locals: {'add': <function add>, 'a': 1, 'b': 'def add(x, y):\\n    return x + y', 'c': <function add>}"
None


In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    b = "def add(x, y):\n    return x + y"
    c = func
    exec(b) # create the new add in locals
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] # this add function only exist in locals()
    print(add1(1,9)) # only from locals()
    print(func(1,9)) # from globals()
    print(c(1,9)) # from globals()
    print(add(1,9)) # from globals()
    print(globals()['add']) 

test(add)

("locals: {'func': <function add>, 'a': 1, 'b': 'def add(x, y):\\n    return x + y', 'c': <function add>, 'add': <function "
 'add at 0x11dcc31f0>}')
10
1
1
1
<function add>


### What would happen if a value outside exec get updated inside exec?

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    exec(b) # create the new add in locals
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] # this add function only exist in locals()
    add1(5,6)
    pprint(f'locals: {locals()}', width=157)

try:
    test(add)
except NameError as e:
    print(e)

("locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x11dcff310>}')
name 'lst' is not defined


### magic of `exec(b, globals().update(locals()))`

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    exec(b, globals().update(locals())) # add the env into exec, so that lst is available
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] 
    print(add1(5,6))
    pprint(f'locals: {locals()}', width=157)

try:
    test(add)
except NameError as e:
    print(e)

("locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x11dcff550>}')
11
("locals: {'func': <function add>, 'a': 1, 'lst': [5], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 "0x11dcff550>, 'add1': <function add>}")


### Mystery: to be solved

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)    
    exec(b, globals().update(locals())) # add the env into exec, so that lst is available

    add1 = locals()['add'] 
    print(add1(5,6))
    add1(5,6)
    pprint(f'locals: {locals()}', width=157)

try:
    test(add)
except NameError as e:
    print(e)

"locals: {'func': <function add>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
11
("locals: {'func': <function add>, 'a': 1, 'lst': [5, 5], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add "
 "at 0x11dcff9d0>, 'add1': <function add>}")


## make a colorful string

In [ ]:
#|export
class dbcolors:
    g = '\033[92m' #GREEN
    y = '\033[93m' #YELLOW
    r = '\033[91m' #RED
    reset = '\033[0m' #RESET COLOR

In [ ]:
#|export
def colorize(cmt, color:str=None):
    if color == "g":
        return dbcolors.g + cmt + dbcolors.reset
    elif color == "y":
        return dbcolors.y + cmt + dbcolors.reset
    elif color == "r":
        return dbcolors.r + cmt + dbcolors.reset
    else: 
        return cmt

In [ ]:
colorize("this is me", "r")

'\x1bthis is me\x1b'

In [ ]:
print(colorize("this is me", "r"))

this is me


## align text to the most right

In [ ]:
#| export
import re

In [ ]:
#| export
def strip_ansi(source):
    return re.sub(r'\033\[(\d|;)+?m', '', source)

In [ ]:
#| export
def alignright(blocks):
    lst = blocks.split('\n')
    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))
    indent = defaults.margin - maxlen
    for l in lst:
        print(' '*indent + format(l))

In [ ]:
alignright("this is me")

                                                                                                                                                   this is me


## print out src code

In [ ]:
#| export
import inspect

### basic version

In [ ]:

def printsrc(src, srclines, cmt):
# print out the title
    print('\n')
    print('{:#^157}'.format(" srcline under investigation "))
    print('\n')


    # convert the source code of the function into a list of strings splitted by '\n'
    lst = inspect.getsource(src).split('\n')

    ccount = 0
    for l in lst:
        if bool(l) and l.strip() in srclines:# print out the srcline under investigation
            print('{:=<157}'.format(l))
            ccount = ccount + 1

            if bool(cmt): # print out comment at the end of the srclines under investigation
                numsrclines = len(srclines.split("\n"))
                if ccount == numsrclines:
                    colcmt = colorize(cmt, "r") # colorize the comment
                    alignright(colcmt) # put the comment to the most right

        else: 
            print('{:<157}'.format(l)) # print out the rest of source code

In [ ]:
def foo():     
    a = 1 + 1
    b = a + 1
    pass

In [ ]:
printsrc(foo, "a = 1 + 1", "this is comment")



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    a = 1 + 1================================================================================================================================================
                                                                                                                                              this is comment
    b = a + 1                                                                                                                                                
    pass                                                                                                                                                     
                                                

In [ ]:
printsrc(foo, "    a = 1 + 1\n    b = a + 1", "this is comment")



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    a = 1 + 1================================================================================================================================================
    b = a + 1================================================================================================================================================
                                                                                                                                              this is comment
    pass                                                                                                                                                     
                                                

### print src with specific number of lines

In [ ]:
#| export
def printsrc(src, srclines, cmt, lines:int=5):
# print out the title
    print('\n')
    print('{:#^157}'.format(" srcline under investigation "))
    print('\n')


    # convert the source code of the function into a list of strings splitted by '\n'
    lst = inspect.getsource(src).split('\n')
    
    # find the idx of the first srcline under investigation in src code
    

    ccount = 0
    for l in lst:
        if bool(l) and l.strip() in srclines:# print out the srcline under investigation with # as padding
            print('{:=<157}'.format(l))
            ccount = ccount + 1

            if bool(cmt): # print out comment at the end of the srclines under investigation
                numsrclines = len(srclines.split("\n"))
                if ccount == numsrclines:
                    colcmt = colorize(cmt, "r") # colorize the comment
                    alignright(colcmt) # put the comment to the most right

        else:
            print('{:<157}'.format(l)) # print out the rest of source code to the most left

    

## dbprint on expression

### basic version

In [ ]:
#| export
def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "debug a srcline with one or more expressions with src printed."
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt)
    
    for c in code:
    # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
        output = f"{c} => {c} : {eval(c, globals().update(env))}"
        print('{:>157}'.format(output))   

In [ ]:
def foo():     
    a = 1 + 1
    b = a + 1
    pass

In [ ]:
def foo(): 
    dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")
    a = 1 + 1
    pass

In [ ]:
foo()



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")                                                                                       
    a = 1 + 1================================================================================================================================================
                                                                                                                                               this is a test
    pass                                                                                                                                                     
                                                

## printrunsrclines() 

It can print out only srclines which actually ran

### Examples

#### simple example

In [ ]:
# def foo(a):
#     if a > 1:
#         a = 1 + 1
#     else:
#         b = a + 1

#### complex example

In [ ]:
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a
        

In [ ]:
foo(3)

3

### insert a line after each srcline to add idx

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""

for idx, l in zip(range(len(srclines)), srclines):
    # if "if" in l or "else" in l or "for" in l:
        
    dbsrc = dbsrc + l + f"\n    srcidx.append({idx})\n" # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

def foo(a):
    srcidx.append(0)
    "this is docs"
    srcidx.append(1)
    # this is a pure comment
    srcidx.append(2)
    if a > 1:
    srcidx.append(3)
        for i in range(3):
    srcidx.append(4)
            a = i + 1
    srcidx.append(5)
    else:
    srcidx.append(6)
        "this is docs"
    srcidx.append(7)
        b = a + 1
    srcidx.append(8)
    "this is docs"
    srcidx.append(9)
    return a
    srcidx.append(10)

    srcidx.append(11)



### add correct indentation to each inserted line

#### count the indentation for each srcline

In [ ]:
len("    a = 1") - len("    a = 1".strip())

4

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"
    dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"   # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

def foo(a):
srcidx.append(0)
    "this is docs"
    srcidx.append(1)
    # this is a pure comment
    srcidx.append(2)
    if a > 1:
    srcidx.append(3)
        for i in range(3):
        srcidx.append(4)
            a = i + 1
            srcidx.append(5)
    else:
    srcidx.append(6)
        "this is docs"
        srcidx.append(7)
        b = a + 1
        srcidx.append(8)
    "this is docs"
    srcidx.append(9)
    return a
    srcidx.append(10)

srcidx.append(11)



### indentation special case: if, else, for, def

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""
indent = 4

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"

    if "if" in l or "else" in l or "for" in l or "def" in l:
        numindent = numindent + indent
    
    dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

def foo(a):
    srcidx.append(0)
    "this is docs"
    srcidx.append(1)
    # this is a pure comment
    srcidx.append(2)
    if a > 1:
        srcidx.append(3)
        for i in range(3):
            srcidx.append(4)
            a = i + 1
            srcidx.append(5)
    else:
        srcidx.append(6)
        "this is docs"
        srcidx.append(7)
        b = a + 1
        srcidx.append(8)
    "this is docs"
    srcidx.append(9)
    return a
    srcidx.append(10)

srcidx.append(11)



### remove pure comments or docs from dbsrc
Do not insert line for pure comment or pure "\n"

In [ ]:
from pprint import pprint
for l in srclines:
    pprint(l)

'def foo(a):'
'    "this is docs"'
'    # this is a pure comment'
'    if a > 1:'
'        for i in range(3):'
'            a = i + 1'
'    else:'
'        "this is docs"'
'        b = a + 1'
'    "this is docs"'
'    return a'
''


In [ ]:
"# this is a comment".startswith("#")

True

In [ ]:
"a = 1 # this is comment".startswith("#")

False

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""
indent = 4

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"

    if "if" in l or "else" in l or "for" in l or "def" in l:
        numindent = numindent + indent
    
    if bool(l): # ignore pure '\n'
        dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

def foo(a):
    srcidx.append(0)
    "this is docs"
    srcidx.append(1)
    # this is a pure comment
    srcidx.append(2)
    if a > 1:
        srcidx.append(3)
        for i in range(3):
            srcidx.append(4)
            a = i + 1
            srcidx.append(5)
    else:
        srcidx.append(6)
        "this is docs"
        srcidx.append(7)
        b = a + 1
        srcidx.append(8)
    "this is docs"
    srcidx.append(9)
    return a
    srcidx.append(10)



In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""
indent = 4

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"

    if "if" in l or "else" in l or "for" in l or "def" in l:
        numindent = numindent + indent
    
    if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
        dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'): # now the dbsrc has no pure comment and pure docs
    print(l)

def foo(a):
    srcidx.append(0)
    if a > 1:
        srcidx.append(3)
        for i in range(3):
            srcidx.append(4)
            a = i + 1
            srcidx.append(5)
    else:
        srcidx.append(6)
        b = a + 1
        srcidx.append(8)
    return a
    srcidx.append(10)



In [ ]:
foo??

Signature: foo(a)
Source:   
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a
File:      /var/folders/gz/ch3n2mp51m9386sytqf97s6w0000gn/T/ipykernel_15219/3208300118.py
Type:      function


In [ ]:
exec(dbsrc) # give life to dbsrc

In [ ]:
foo??

Signature: foo(a)
Docstring: <no docstring>
File:      Dynamically generated function. No source code available.
Type:      function


In [ ]:
srcidx = [] #used outside the srcode

In [ ]:
foo(3) # run the example using dbsrc
# foo(-1) # run the example using dbsrc
srcidx # Now it should have all the idx whose srclines have run

[0, 3, 4, 5, 4, 5, 4, 5]

### print out the srclines which get run

In [ ]:
for idx, l in zip(range(len(srclines)), srclines):
    if idx in srcidx:
        print(l)

def foo(a):
    if a > 1:
        for i in range(3):
            a = i + 1


### Make sure all if, else, for get printed

In [ ]:
for idx, l in zip(range(len(srclines)), srclines):
    if idx in srcidx or "for" in l or "if" in l or "else" in l:
        print(l)

def foo(a):
    if a > 1:
        for i in range(3):
            a = i + 1
    else:


### Put all together into the function printrunsrclines()

In [ ]:
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a
        

In [ ]:
def printrunsrclines(func):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if" in l or "else" in l or "for" in l or "def" in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line
    
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))
    fool = locals()['foo']
    pprint(fool(3))
    pprint(locals())

     
    # run = "foo(3)"
    exec("fool(3)")
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(foo)

3
{'addline': 'srcidx.append(11)',
 'dbsrc': 'def foo(a):\n'
          '    srcidx.append(0)\n'
          '    if a > 1:\n'
          '        srcidx.append(3)\n'
          '        for i in range(3):\n'
          '            srcidx.append(4)\n'
          '            a = i + 1\n'
          '            srcidx.append(5)\n'
          '    else:\n'
          '        srcidx.append(6)\n'
          '        b = a + 1\n'
          '        srcidx.append(8)\n'
          '    return a\n'
          '    srcidx.append(10)\n',
 'foo': <function foo>,
 'fool': <function foo>,
 'func': <function foo>,
 'idx': 11,
 'indent': 4,
 'l': '',
 'numindent': 0,
 'srcidx': [0, 3, 4, 5, 4, 5, 4, 5],
 'srclines': ['def foo(a):',
              '    "this is docs"',
              '    # this is a pure comment',
              '    if a > 1:',
              '        for i in range(3):',
              '            a = i + 1',
              '    else:',
              '        "this is docs"',
              '     

#### no more renaming of foo

In [ ]:
def printrunsrclines(func):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if" in l or "else" in l or "for" in l or "def" in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line
    
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec("foo(3)") # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
def foo(a):

    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        b = a + 1
    return a

In [ ]:
printrunsrclines(foo)

[0, 2, 3, 4, 3, 4, 3, 4]
def foo(a):
    if a > 1:
        for i in range(3):
            a = i + 1
    else:


#### add example as a param into the function

In [ ]:
#| export
def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if" in l or "else" in l or "for" in l or "def" in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(foo, "foo(-1)")

('def foo(a):\n'
 '    srcidx.append(0)\n'
 '    if a > 1:\n'
 '        srcidx.append(2)\n'
 '        for i in range(3):\n'
 '            srcidx.append(3)\n'
 '            a = i + 1\n'
 '            srcidx.append(4)\n'
 '    else:\n'
 '        srcidx.append(5)\n'
 '        b = a + 1\n'
 '        srcidx.append(6)\n'
 '    return a\n'
 '    srcidx.append(7)\n')
[0, 5, 6]
def foo(a):
    if a > 1:
        for i in range(3):
    else:
        b = a + 1


In [ ]:
printrunsrclines(foo, "foo(2)")

('def foo(a):\n'
 '    srcidx.append(0)\n'
 '    if a > 1:\n'
 '        srcidx.append(2)\n'
 '        for i in range(3):\n'
 '            srcidx.append(3)\n'
 '            a = i + 1\n'
 '            srcidx.append(4)\n'
 '    else:\n'
 '        srcidx.append(5)\n'
 '        b = a + 1\n'
 '        srcidx.append(6)\n'
 '    return a\n'
 '    srcidx.append(7)\n')
[0, 2, 3, 4, 3, 4, 3, 4]
def foo(a):
    if a > 1:
        for i in range(3):
            a = i + 1
    else:


#### improve on search for `if`, else, for, def to avoid errors for more examples

In [ ]:

def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if " in l or "else:" in l or "for " in l or "def " in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') \
        and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(alignright, 'alignright("this is me")')

#### remove an empty line with indentation

In [ ]:
lst = """
this is code\n\
     \n\
this is code
""".split('\n')
print(lst)
for l in lst:
    print(bool(l.strip()))

['', 'this is code', '     ', 'this is code', '']
False
True
False
True
False


In [ ]:

def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip()) # how to strip only the left not the right?????
        addline = f"srcidx.append({idx})"

        if "if " in l or "else:" in l or "for " in l or "def " in l:
            numindent = numindent + indent

        if bool(l.strip()) and not l.strip().startswith('#') \
        and not (l.strip().startswith('"') and l.strip().endswith('"')): 
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(alignright, 'alignright("this is me")')

('def alignright(blocks):\n'
 '    srcidx.append(0)\n'
 "    lst = blocks.split('\\n')\n"
 '    srcidx.append(1)\n'
 '    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))\n'
 '    srcidx.append(2)\n'
 '    indent = defaults.margin - maxlen\n'
 '    srcidx.append(3)\n'
 '    for l in lst:\n'
 '        srcidx.append(4)\n'
 "        print(' '*indent + format(l))\n"
 '        srcidx.append(5)\n')
                                                                                                                                                   this is me
[0, 1, 2, 3, 4, 5]
def alignright(blocks):
    lst = blocks.split('\n')
    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))
    indent = defaults.margin - maxlen
    for l in lst:
        print(' '*indent + format(l))


In [ ]:
pprint(inspect.getsource(printsrc))

('def printsrc(src, srclines, cmt, lines:int=5):\n'
 '# print out the title\n'
 "    print('\\n')\n"
 '    print(\'{:#^157}\'.format(" srcline under investigation "))\n'
 "    print('\\n')\n"
 '\n'
 '\n'
 '    # convert the source code of the function into a list of strings '
 "splitted by '\\n'\n"
 "    lst = inspect.getsource(src).split('\\n')\n"
 '    \n'
 '    # find the idx of the first srcline under investigation in src code\n'
 '    \n'
 '\n'
 '    ccount = 0\n'
 '    for l in lst:\n'
 '        if bool(l) and l.strip() in srclines:# print out the srcline under '
 'investigation with # as padding\n'
 "            print('{:=<157}'.format(l))\n"
 '            ccount = ccount + 1\n'
 '\n'
 '            if bool(cmt): # print out comment at the end of the srclines '
 'under investigation\n'
 '                numsrclines = len(srclines.split("\\n"))\n'
 '                if ccount == numsrclines:\n'
 '                    colcmt = colorize(cmt, "r") # colorize the comment\n'
 '          

In [ ]:
#| export
def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip()) # how to strip only the left not the right?????
        addline = f"srcidx.append({idx})"

        if "if " in l or "else:" in l or "for " in l or "def " in l:
            numindent = numindent + indent

        if bool(l.strip()) and not l.strip().startswith('#') \
        and not (l.strip().startswith('"') and l.strip().endswith('"')): 
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    # pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    # print(srcidx)

    # pprint(srclines)
    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(printsrc, 'printsrc(foo, "    else:\\n    b = a + 1", "this is comment")') # make sure to use \\n not \n



################################################################ srcline under investigation ################################################################


def foo(a):                                                                                                                                                  
                                                                                                                                                             
    if a > 1:                                                                                                                                                
        for i in range(3):                                                                                                                                   
            a = i + 1                                                                                                                                        
    else:=======================================

In [ ]:
printrunsrclines(printsrc, 'printsrc(foo, "    else:\\n    b = a + 1", "")') # make sure to use \\n not \n



################################################################ srcline under investigation ################################################################


def foo(a):                                                                                                                                                  
                                                                                                                                                             
    if a > 1:                                                                                                                                                
        for i in range(3):                                                                                                                                   
            a = i + 1                                                                                                                                        
    else:=======================================

### more difficult examples to test printrunsrc()

#|hide
## Export

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()

#|hide
## Send to Obsidian

In [ ]:
#| hide
!jupytext --to md /Users/Natsume/Documents/fastdebug/00_core.ipynb
!mv /Users/Natsume/Documents/fastdebug/00_core.md \
/Users/Natsume/Documents/divefastai/Debuggable/jupytext/

In [ ]:
#| hide
!jupyter nbconvert --config /Users/Natsume/Documents/mynbcfg.py --to markdown \
--output-dir /Users/Natsume/Documents/divefastai/Debuggable/nbconvert